## Dataset
The dataset is taken from the ISIC (International Skin Image Collaboration) Archive. It consists of 1800 pictures of benign moles and 1497 pictures of malignant classified moles. The pictures have all been resized to low resolution (224x224x3) RGB. The task of this kernel is to create a model, which can classify a mole visually into benign and malignant. 

As the dataset is pretty balanced, the model will be tested on the accuracy score, thus (TP + TN)/(ALL).

It has 2 different classes of skin cancer which are listed below :<br>
**1. Benign <br>**
**2. Malignant <br>**

In [1]:
%matplotlib inline

import os
import gc
from PIL import Image

import tqdm
import numpy as np
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, LeakyReLU
from keras.applications import VGG16
from tensorflow import set_random_seed
from sklearn.utils import check_random_state


sns.set()
np.random.seed(0);
set_random_seed(0);
check_random_state(0);

Using TensorFlow backend.


## data upload

The data consists of two folders of the two types of moles.

In [2]:

#
# Labels:
# 0 -> benign
# 1 -> malignant

train_imgs, test_imgs = [], []
train_labels, test_labels = [], []

for img_path in os.listdir('../input/data/train/benign'):
    train_imgs.append('../input/data/train/benign/' + img_path)
    train_labels.append(0)
    
for img_path in os.listdir('../input/data/train/malignant'):
    train_imgs.append('../input/data/train/malignant/' + img_path)
    train_labels.append(1)
    
for img_path in os.listdir('../input/data/test/benign'):
    test_imgs.append('../input/data/test/benign/' + img_path)
    test_labels.append(0)
    
for img_path in os.listdir('../input/data/test/malignant'):
    test_imgs.append('../input/data/test/malignant/' + img_path)
    test_labels.append(1)
    
train_imgs, test_imgs = np.array(train_imgs), np.array(test_imgs)
train_labels, test_labels = np.array(train_labels), np.array(test_labels)
    
class_distribution = np.bincount(np.concatenate([train_labels, test_labels]))
    
print('Size of train set:', len(train_imgs))
print('Size of test set:', len(test_imgs))
print(class_distribution[0], 'benign labeled samples and', class_distribution[1], 'malignant')

Size of train set: 2637
Size of test set: 660
1800 benign labeled samples and 1497 malignant


## Data selection
the whole set contains 3297 images. We split data so that test set is 20%, 15% is for val data.

In [3]:
# Load the images to memory
xtrain, xtest = [], []
ytrain, ytest = train_labels, test_labels

for filename in tqdm.tqdm(train_imgs):
    xtrain.append(np.array(Image.open(filename)))
    
for filename in tqdm.tqdm(test_imgs):
    xtest.append(np.array(Image.open(filename)))
    
del train_imgs, test_imgs, train_labels, test_labels
xtrain, xtest = np.array(xtrain), np.array(xtest)

# Merge and split train and test set to have more train data
data = np.concatenate([xtrain, xtest])
labels = np.concatenate([ytrain, ytest])

# Spliting data to train, validation and test values
xtrain, xtest, ytrain, ytest = train_test_split(data, labels, test_size=.2, random_state=0)
xtra, xval, ytra, yval = train_test_split(xtrain, ytrain, test_size=.2, random_state=0, shuffle=False)

gc.collect()
print('Shape of the new train set:', xtra.shape)
print('Shape of the new test set:', xtest.shape)
print('Shape of the validation set:', xval.shape)

100%|██████████| 660/660 [00:02<00:00, 323.32it/s]


Shape of the new train set: (2109, 224, 224, 3)
Shape of the new test set: (660, 224, 224, 3)
Shape of the validation set: (528, 224, 224, 3)


then we add augmented images to train data.

In [5]:
data_generator = ImageDataGenerator(rotation_range=90,
                                    width_shift_range=0.15,
                                    height_shift_range=0.15,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    brightness_range=[0.8, 1.1],
                                    fill_mode='nearest')

In [6]:
new_samples, new_labels = next(data_generator.flow(xtra, ytra, batch_size=len(xtra)))
xtra = np.concatenate([xtra, new_samples])
ytra = np.concatenate([ytra, new_labels])

del new_samples, new_labels
print('New number of training samples:', len(xtra))

New number of training samples: 4218


Use the simple normalization technique where we just divide each pixel value by `255`.

In [7]:
# Normalizing values
xtra = xtra.astype('float32') / 255.
xtest = xtest.astype('float32') / 255.
xval = xval.astype('float32') / 255.

print('Training data shape:', xtra.shape)
print('Min value:', xtra.min())
print('Max value:', xtra.max())

Training data shape: (4218, 224, 224, 3)
Min value: 0.0
Max value: 1.0


In [8]:
print('Test data shape:', xtest.shape)
print('Val data shape:', xval.shape)

Val data shape: (528, 224, 224, 3)


In [9]:
del data

In [10]:
del labels

In [12]:
X_tmp=np.concatenate([xtra, xtest,xval])
Y_tmp=np.concatenate([ytra, ytest,yval])
print(' data shape:', X_tmp.shape)

Val data shape: (5406, 224, 224, 3)


In [13]:
del xval,xtest,xtra,ytra,ytest,yval

In [18]:
train_x, test_x,train_y,test_y=train_test_split(X_tmp, Y_tmp, test_size=.2, random_state=101)

## Building and training the model



In [19]:

# Build the model
model = Sequential()

model.add(VGG16(include_top=False, input_shape=(224, 224, 3,)))
model.add(Flatten())
model.add(Dense(32))
model.add(LeakyReLU(0.001))
model.add(Dense(16))
model.add(LeakyReLU(0.001))
model.add(Dense(2, activation='softmax'))
model.layers[0].trainable = False

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                802848    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 16)                528       
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 34        
Total para

### train the model.

##### testing another way of splitting data. Use chanks under #. This is just for testing.

In [20]:
# Train the model
N_EPOCHS = 30
h = model.fit(train_x, train_y, validation_split=0.2, epochs=N_EPOCHS, batch_size=64,shuffle=True)

Train on 3459 samples, validate on 865 samples
Epoch 1/30
3459/3459 [==============================] - 12s 4ms/step - loss: 0.5622 - acc: 0.7395 - val_loss: 0.4137 - val_acc: 0.8012
Epoch 2/30
3459/3459 [==============================] - 11s 3ms/step - loss: 0.3514 - acc: 0.8439 - val_loss: 0.3951 - val_acc: 0.8335
Epoch 3/30
3459/3459 [==============================] - 11s 3ms/step - loss: 0.3278 - acc: 0.8583 - val_loss: 0.5612 - val_acc: 0.7283
Epoch 4/30
3459/3459 [==============================] - 11s 3ms/step - loss: 0.3028 - acc: 0.8690 - val_loss: 0.3889 - val_acc: 0.8335
Epoch 5/30
3459/3459 [==============================] - 11s 3ms/step - loss: 0.2563 - acc: 0.8907 - val_loss: 0.3814 - val_acc: 0.8405
Epoch 6/30
3459/3459 [==============================] - 11s 3ms/step - loss: 0.2468 - acc: 0.8962 - val_loss: 0.4390 - val_acc: 0.8139
Epoch 7/30
3459/3459 [==============================] - 11s 3ms/step - loss: 0.2574 - acc: 0.8849 - val_loss: 0.3906 - val_acc: 0.8358
Epoch 8/

In [21]:
print('Accuracy on test set:', model.evaluate(test_x, test_y)[1])

1082/1082 [==============================] - 3s 3ms/step
Accuracy on test set: 0.8373382629175944


In [ ]:
# Train the model
#N_EPOCHS = 30
#h = model.fit(xtra, ytra, validation_data=(xval, yval), epochs=N_EPOCHS, batch_size=64,shuffle=True)

Visualizing the accuracy and losses to check if the model is overfit.

In [ ]:
# Plotting accuracy history
plt.figure(figsize=(15, 8))
plt.scatter(range(N_EPOCHS), h.history['acc'], marker='x', label='Training accuracy');
plt.plot(range(N_EPOCHS), h.history['val_acc'], color='green', label='Validation accuracy');
plt.legend();
plt.title('Accuracy');

# Plotting loss history
plt.figure(figsize=(15, 8))
plt.scatter(range(N_EPOCHS), h.history['loss'], marker='x', label='Training loss');
plt.plot(range(N_EPOCHS), h.history['val_loss'], color='green', label='Validation loss');
plt.legend();
plt.title('Loss');

Evaluate the model on test data to double check it's accuracy, and save it.

In [ ]:
print('Accuracy on test set:', model.evaluate(xtest, ytest)[1])
#model.save('skin_cancer_final.h5')

## Cross-validation:
    

In [16]:

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kf = KFold(n_splits=5) # Define the split - into 5 folds 

kf.get_n_splits(X_tmp) # returns the number of splitting iterations in the cross-validator
scores=[]
for train_index, test_index in kf.split(X_tmp):
    X_train, X_test = X_tmp[train_index], X_tmp[test_index]
    y_train, y_test = Y_tmp[train_index], Y_tmp[test_index]
    model.fit(X_train,y_train,epochs=30, batch_size=64,shuffle=True)
    score=model.evaluate(X_test,y_test)[1]
    print(score)
    scores.append(score)
    del X_train, X_test, y_train, y_test



Epoch 1/30
4324/4324 [==============================] - 18s 4ms/step - loss: 0.5680 - acc: 0.7359
Epoch 2/30
4324/4324 [==============================] - 13s 3ms/step - loss: 0.3768 - acc: 0.8340
Epoch 3/30
4324/4324 [==============================] - 14s 3ms/step - loss: 0.3250 - acc: 0.8550
Epoch 4/30
4324/4324 [==============================] - 13s 3ms/step - loss: 0.2817 - acc: 0.8742
Epoch 5/30
4324/4324 [==============================] - 14s 3ms/step - loss: 0.2491 - acc: 0.8934
Epoch 6/30
4324/4324 [==============================] - 14s 3ms/step - loss: 0.2538 - acc: 0.8888
Epoch 7/30
4324/4324 [==============================] - 13s 3ms/step - loss: 0.2144 - acc: 0.9052
Epoch 8/30
4324/4324 [==============================] - 13s 3ms/step - loss: 0.1888 - acc: 0.9191
Epoch 9/30
4324/4324 [==============================] - 13s 3ms/step - loss: 0.2182 - acc: 0.9022
Epoch 10/30
4324/4324 [==============================] - 14s 3ms/step - loss: 0.1608 - acc: 0.9341
Epoch 11/30
4324/43

In [17]:
np.array(scores).mean()

0.9534026650117617